# Лабораторна робота 3
**Тема:** Лінійна регресія

**Автор:** Herasymenko  •  **Варіант:** 1  •  **Дата:** 2025-10-28

**Важливо:** Кожен підпункт виконаний в окремій комірці з коментарем-умовою.

In [ ]:
# === Імпорт бібліотек ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

pd.set_option('display.max_columns', 50)
print('✅ Імпорт виконано')

## 🧠 Завдання 1. Diabetes Dataset

### 1. Завантажити Diabetes Dataset та вивести перших 5 рядків

In [ ]:
# Умова: Завантажити датасет і показати перші 5 рядків
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target
df.head()

### 2. Перевірити пропуски; при наявності — замінити на середнє

In [ ]:
# Умова: Перевірити пропуски і при потребі заповнити середнім
na_before = df.isna().sum()
for c in df.columns:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].mean())
na_after = df.isna().sum()
na_before, na_after

### 3. Перевірити дублікати; видалити при наявності

In [ ]:
# Умова: Перевірити дублікати і видалити
dups = df.duplicated().sum()
df = df.drop_duplicates()
dups, df.shape

### 4. Замінити бінарні ознаки; перевірити унікальні значення «sex»

In [ ]:
# Умова: Перевірити унікальні значення у стовпці 'sex'
if 'sex' in df.columns:
    uniques_sex = df['sex'].unique()
    # У цьому датасеті всі ознаки вже числові й масштабовані; додаткове кодування не потрібне
    uniques_sex
else:
    'Стовпця sex немає'

### 5. Перевірити типи даних; за потреби конвертувати до числових

In [ ]:
# Умова: Перевірити типи даних; усе має бути числовим
dtypes_info = df.dtypes
# Якщо раптом щось нечислове, спробуємо привести
for c in df.columns:
    if df[c].dtype == 'O':
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(df[c].dropna().astype(str).str.extract(r'([\-\d\.]+)')[0].astype(float).mean() if df[c].dropna().shape[0]>0 else 0)
dtypes_info

### 6. Обчислити кореляцію з target; відсортувати; побудувати теплову карту

In [ ]:
# Умова: Кореляція з цільовою змінною та heatmap
corr_series = df.corr(numeric_only=True)['target'].sort_values(ascending=False)
corr_series

In [ ]:
# Теплова карта кореляції (matplotlib)
corr_mat = df.corr(numeric_only=True)
plt.figure(figsize=(8,6))
im = plt.imshow(corr_mat, aspect='auto')
plt.colorbar(im)
plt.xticks(range(len(corr_mat.columns)), corr_mat.columns, rotation=90)
plt.yticks(range(len(corr_mat.index)), corr_mat.index)
plt.title('Кореляційна матриця (Diabetes)')
plt.tight_layout()
plt.show()

### 7. Масштабувати ознаки

In [ ]:
# Умова: Масштабування ознак (окрім target)
X = df.drop('target', axis=1)
y = df['target']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X.shape, X_scaled.shape

### 8. Поділити дані на тренувальну та тестову вибірки

In [ ]:
# Умова: Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

### 9. Побудувати Linear Regression і RandomForest на найбільш корельованих ознаках (підібрати k)

In [ ]:
# Умова: Вибрати k найбільш корельованих ознак
k = 6  # можна змінити й перевірити
top_features = corr_series.drop('target').abs().sort_values(ascending=False).head(k).index.tolist()
top_features

In [ ]:
# Навчання моделей лише на top_features
feat_idx = [list(X.columns).index(f) for f in top_features]
X_train_top = X_train[:, feat_idx]
X_test_top  = X_test[:, feat_idx]

# Лінійна регресія
lin = LinearRegression().fit(X_train_top, y_train)
y_pred_lin = lin.predict(X_test_top)

# RandomForest (підберемо n_estimators грубо)
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train_top, y_train)
y_pred_rf = rf.predict(X_test_top)

'Навчено моделі на top_features'

### 10. Вивести R² і MSE для обох моделей

In [ ]:
# Метрики
metrics = pd.DataFrame({
    'Model': ['LinearRegression','RandomForest'],
    'R2': [r2_score(y_test, y_pred_lin), r2_score(y_test, y_pred_rf)],
    'MSE': [mean_squared_error(y_test, y_pred_lin), mean_squared_error(y_test, y_pred_rf)]
})
metrics

### 11. Побудувати графіки (y_true vs y_pred)

In [ ]:
# Порівняння прогнозів
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(y_test, y_pred_lin, alpha=0.7)
plt.xlabel('Справжні значення'); plt.ylabel('Прогноз (Linear)'); plt.title('Linear Regression')

plt.subplot(1,2,2)
plt.scatter(y_test, y_pred_rf, alpha=0.7)
plt.xlabel('Справжні значення'); plt.ylabel('Прогноз (RF)'); plt.title('Random Forest')

plt.tight_layout(); plt.show()

### 12. Вивести справжні і прогнозовані значення

In [ ]:
res = pd.DataFrame({'Actual': y_test.reset_index(drop=True),
                    'Pred_Linear': pd.Series(y_pred_lin),
                    'Pred_RF': pd.Series(y_pred_rf)})
res.head(10)

### 13. ВИСНОВКИ (заповни у Markdown-клітинці нижче)

**Висновок до Завдання 1:**
- Які ознаки найсильніше корелюють із target?
- Яка модель показала кращий R² / MSE?
- Чи допоміг відбір найбільш корельованих ознак?


---
## 🏠 Завдання 2. California Housing Dataset

### 1. Завантажити датасет і вивести перших 5 рядків

In [ ]:
# Умова: Завантажити California Housing (вбудований у sklearn)
from sklearn.datasets import fetch_california_housing
cal = fetch_california_housing(as_frame=True)
df2 = cal.frame.copy()
df2.head()

### 2. Попередній аналіз: пропуски, дублікати, типи, кореляція + heatmap

In [ ]:
# Пропуски
na2_before = df2.isna().sum()
for c in df2.columns:
    if df2[c].isna().any():
        df2[c] = df2[c].fillna(df2[c].mean())
na2_after = df2.isna().sum()
na2_before, na2_after

In [ ]:
# Дублікати
dups2 = df2.duplicated().sum()
df2 = df2.drop_duplicates()
dups2, df2.shape

In [ ]:
# Типи даних
df2.dtypes

In [ ]:
# Кореляція цілі (MedHouseVal) з ознаками
target2 = 'MedHouseVal'
corr2 = df2.corr(numeric_only=True)[target2].sort_values(ascending=False)
corr2

In [ ]:
# Heatmap (matplotlib)
corr_mat2 = df2.corr(numeric_only=True)
plt.figure(figsize=(8,6))
im = plt.imshow(corr_mat2, aspect='auto')
plt.colorbar(im)
plt.xticks(range(len(corr_mat2.columns)), corr_mat2.columns, rotation=90)
plt.yticks(range(len(corr_mat2.index)), corr_mat2.index)
plt.title('Кореляційна матриця (California)')
plt.tight_layout(); plt.show()

### 3. Масштабувати ознаки

In [ ]:
X2 = df2.drop(columns=[target2])
y2 = df2[target2]
scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)
X2.shape, X2_scaled.shape

### 4. Поділити дані на тренувальні/тестові

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.2, random_state=42)
X2_train.shape, X2_test.shape

### 5. Побудувати 3 моделі (Linear, RandomForest, Ridge) з підбором параметрів (GridSearchCV)

In [ ]:
# Лінійна регресія (без параметрів для гріду — просто базова) 
lin2 = LinearRegression()
lin2.fit(X2_train, y2_train)
y2_pred_lin = lin2.predict(X2_test)

# RandomForest з GridSearchCV
rf2 = RandomForestRegressor(random_state=42)
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20]
}
gs_rf = GridSearchCV(rf2, param_grid_rf, cv=3, scoring='r2', n_jobs=-1)
gs_rf.fit(X2_train, y2_train)
best_rf = gs_rf.best_estimator_
y2_pred_rf = best_rf.predict(X2_test)

# Ridge з GridSearchCV
ridge = Ridge()
param_grid_rg = {'alpha': [0.1, 1.0, 10.0]}
gs_rg = GridSearchCV(ridge, param_grid_rg, cv=3, scoring='r2', n_jobs=-1)
gs_rg.fit(X2_train, y2_train)
best_ridge = gs_rg.best_estimator_
y2_pred_ridge = best_ridge.predict(X2_test)

gs_rf.best_params_, gs_rg.best_params_

### 6. Вивести R² і MSE для трьох моделей; побудувати графіки

In [ ]:
metrics2 = pd.DataFrame({
    'Model': ['Linear','RandomForest','Ridge'],
    'R2': [r2_score(y2_test, y2_pred_lin), r2_score(y2_test, y2_pred_rf), r2_score(y2_test, y2_pred_ridge)],
    'MSE': [mean_squared_error(y2_test, y2_pred_lin), mean_squared_error(y2_test, y2_pred_rf), mean_squared_error(y2_test, y2_pred_ridge)]
})
metrics2

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.scatter(y2_test, y2_pred_lin, alpha=0.6); plt.title('Linear')
plt.xlabel('Actual'); plt.ylabel('Predicted')

plt.subplot(1,3,2)
plt.scatter(y2_test, y2_pred_rf, alpha=0.6); plt.title('RandomForest')
plt.xlabel('Actual'); plt.ylabel('Predicted')

plt.subplot(1,3,3)
plt.scatter(y2_test, y2_pred_ridge, alpha=0.6); plt.title('Ridge')
plt.xlabel('Actual'); plt.ylabel('Predicted')

plt.tight_layout(); plt.show()

### 7. Вивести справжні і прогнозовані значення

In [ ]:
res2 = pd.DataFrame({
    'Actual': y2_test.reset_index(drop=True),
    'Pred_Linear': pd.Series(y2_pred_lin),
    'Pred_RF': pd.Series(y2_pred_rf),
    'Pred_Ridge': pd.Series(y2_pred_ridge)
})
res2.head(10)

### ВИСНОВКИ (заповни у Markdown-клітинці нижче)

**Висновок до Завдання 2:**
- Які параметри підібрали для RF та Ridge?
- Яка модель показала кращий R² / MSE і чому?
- Чи вплинуло масштабування на якість?